# Отбор факторов в уравнение множественной регрессии 

In [4]:
#импорт бибилиотеки
import pandas as pd

#импорт данных
df = pd.read_excel('Economy_Indicators.xlsx')

#просмотр данных
df.head()

,Country,GDP Year-over-Year,GDP Quarter-over-Quarterr,Interest Rate,Inflation Rate,Jobless Rate,"Gov, Budget",Debt/GDP,Current Account,GDP/P
0,Euro Area,4.1,0.8,1.25,10.0,6.6,-5.1,95.6,2.5,42.307917
1,Germany,1.7,0.1,1.25,10.0,5.5,-3.7,69.3,7.4,50.781626
2,United Kingdom,4.4,0.2,2.25,9.9,3.5,-6.0,95.9,-2.6,47.193840
3,France,4.2,0.5,1.25,5.6,7.4,-6.5,112.9,0.4,43.427473
4,Italy,5.0,1.1,1.25,8.9,7.8,-7.2,150.8,2.5,35.449021


Определим, что означает каждая строка
GDP Quarter-over-Quarter - ВВП по сравнению с предыдущим кварталом
Interest Rate - Процентная ставка
Inflation Rate - Инфляция
Jobless Rate - Уровень безработицы
Gov. Budget - Государственный бюджет
Debt/GDP - Долг/ВВП
Current Account - Текущий счет (сальдо торгового баланса)
GDP/P - ВВП на душу населения

В данном случае выберем GDP/P (ВВП на душу населения) зависимой переменной, так как она лучше всего отражает рост экономики с течением времени. Факторами будут процентная ставка (Interest rate), уровень безработицы (Jobless rate) и уровень инфляции (Inflation rate)

In [5]:
#удалим ненужные столбцы
df = df.drop(['GDP Year-over-Year', 'GDP Quarter-over-Quarterr', 'Gov, Budget', 'Debt/GDP', 'Current Account'], axis=1)
df.head()

,Country,Interest Rate,Inflation Rate,Jobless Rate,GDP/P
0,Euro Area,1.25,10.0,6.6,42.307917
1,Germany,1.25,10.0,5.5,50.781626
2,United Kingdom,2.25,9.9,3.5,47.193840
3,France,1.25,5.6,7.4,43.427473
4,Italy,1.25,8.9,7.8,35.449021


# Оценка параметров уравнения множественной регрессии

In [6]:
#импортируем нужную библиотеку
import statsmodels.api as sm
#зависимая переменная
dependent_variable = 'GDP/P'
#факторы
independent_variables = ['Interest Rate', 'Inflation Rate', 'Jobless Rate']

#добавим константы к факторам
X = sm.add_constant(df[independent_variables])

#выделим зависимую переменную
y = df[dependent_variable]

#создадим модель множественной регрессии
model = sm.OLS(y, X)

#получим результаты оценки модели
results = model.fit()

#выведем статистики оценки параметров
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                  GDP/P   R-squared:                       0.335
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     6.208
Date:                Mon, 06 May 2024   Prob (F-statistic):            0.00159
Time:                        23:58:48   Log-Likelihood:                -188.55
No. Observations:                  41   AIC:                             385.1
Df Residuals:                      37   BIC:                             391.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             65.0125      8.116      8.

По полученному регрессионному анализу можем составить множественное уравнение регрессии: \
**y = -1,66x_1 - 0,5x_2 - 2,15x_3 + 65,01**

Таким образом, можем сделать следующие выводы:
1) С увеличением процентной ставки на 1 п.п. ВВП на душу населения в среднем уменьшается на 1,66 тыс. долларов при неизменных прочих факторах.
2) С увеличением инфляции на 1% ВВП на душу населения в среднем уменьшается на 0,5 тыс. долларов при неизменных прочих факторах.
3) С увеличением безработицы на 1% ВВП на душу населения в среднем уменьшается на 2,15 тыс. долларов при неизменных прочих факторах.

# Стандартизованные коэффициенты регрессии

In [7]:
#импортируем нужную библиотеку
from sklearn.preprocessing import StandardScaler

#стандартизация факторов
scaler = StandardScaler()
X_standardized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

#создадим модель множественной регрессии с стандартизованными данными
model = sm.OLS(y, X_standardized)

#получим результаты оценки модели
results = model.fit()

#выведем статистики оценки параметров
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                  GDP/P   R-squared (uncentered):                   0.133
Model:                            OLS   Adj. R-squared (uncentered):              0.065
Method:                 Least Squares   F-statistic:                              1.944
Date:                Mon, 06 May 2024   Prob (F-statistic):                       0.139
Time:                        23:58:48   Log-Likelihood:                         -212.89
No. Observations:                  41   AIC:                                      431.8
Df Residuals:                      38   BIC:                                      436.9
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------

C:\Users\mrrr_\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1965: RuntimeWarning: divide by zero encountered in scalar divide
  return np.sqrt(eigvals[0]/eigvals[-1])


Получим стандартизованное уравнение регрессии:\
**t_y = -9,01t_x1 - 6,20t_x2 - 10,09t_x3**

Таким образом, можем сделать следующие выводы:
1) С увеличением процентной ставки на 1 СКО ВВП на душу населения в среднем уменьшается на 9,01 тыс. долларов при неизменных прочих факторах.
2) С увеличением инфляции на 1 СКО ВВП на душу населения в среднем уменьшается на 6,2 тыс. долларов при неизменных прочих факторах.
3) С увеличением безработицы на 1 СКО ВВП на душу населения в среднем уменьшается на 10,09 тыс. долларов при неизменных прочих факторах.

# Коэффициенты эластичности

In [8]:
#из модели получим коэффициенты эластичности
elasticities = results.params * (X.mean() / y.mean())

#выведем коэффициенты эластичности
print(elasticities)

const             0.000000
Interest Rate    -0.996259
Inflation Rate   -2.563893
Jobless Rate     -1.884655
dtype: float64


Таким образом, можем сделать следующие выводы:

1) С увеличением процентной ставки на 1% от среднего уровня, ВВП на душу населения в среднем уменьшается на 1% от своего среднего уроня при неизменных прочих факторах.
2) С увеличением инфляции на 1% от среднего уровня, ВВП на душу населения в среднем уменьшается на 2,56% от своего среднего уровня при неизменных прочих факторах.
3) С увеличением безработицы на 1% от среднего уровня, ВВП на душу населения в среднем уменьшается на 1,88% от своего среднего уроня при неизменных прочих факторах.

# Оценка значимости параметров и уравнения множественной регрессии

In [9]:
#выведем статистики оценки параметров и уравнения
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                  GDP/P   R-squared (uncentered):                   0.133
Model:                            OLS   Adj. R-squared (uncentered):              0.065
Method:                 Least Squares   F-statistic:                              1.944
Date:                Mon, 06 May 2024   Prob (F-statistic):                       0.139
Time:                        23:58:48   Log-Likelihood:                         -212.89
No. Observations:                  41   AIC:                                      431.8
Df Residuals:                      38   BIC:                                      436.9
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------

Для оценки значимости уравнения регрессии в целом сравним фактическое и критическое значение F-критерия. Fфакт=1,94, Fкрит определяется как и для парной регрессии. В полученной модели Fкрит=2,85. Таким образом, сравнивая Fфакт и Fкрит приходим к выводу о необходимости принять гипотезу (Н0) и с вероятностью 95% сделать вывод о **статистической незначимости уравнения регрессии**.\
Значимость коэффициентов регрессии полученного уравнения оценим с помощью t-критериев Стьюдента. Если tфакт > tтабл, то коэффициент регрессии является статистически значимым, надежным, на него можно опираться в анализе и прогнозе.
tтабл определяется для принятого уровня значимости α=0,05 (так как по условию заданный уровень вероятности составляет 0,95) и числа степеней свободы df=n-m-1=41-3-1=37\
В полученной модели tтабл = 2,02.\
|tb1|<tтабл, (1,088<2.02)\
|tb2|<tтабл (0,74<2.02)\
|tb3|<tтабо (1,41<2.02)\
Таким образом, коэффициенты регрессии **не является статистически значимыми**

# Оценка значимости включения переменных в уравнение регрессии

In [2]:
#создадим подмодели, исключая каждый из факторов по очереди
reduced_X1 = sm.add_constant(df[['Inflation Rate', 'Jobless Rate']])
reduced_X2 = sm.add_constant(df[['Interest Rate', 'Jobless Rate']])
reduced_X3 = sm.add_constant(df[['Interest Rate', 'Inflation Rate']])

#создадим подмодели
reduced_model_X1 = sm.OLS(y, reduced_X1)
reduced_model_X2 = sm.OLS(y, reduced_X2)
reduced_model_X3 = sm.OLS(y, reduced_X3)

#получим результаты оценки каждой подмодели
reduced_results_X1 = reduced_model_X1.fit()
reduced_results_X2 = reduced_model_X2.fit()
reduced_results_X3 = reduced_model_X3.fit()

NameError: name 'sm' is not defined

In [3]:
#рассчитаем частные F-критерии
partial_F_X1 = (results.ssr - reduced_results_X1.ssr) / 2 / (reduced_results_X1.ssr / (len(df) - 4))
partial_F_X2 = (results.ssr - reduced_results_X2.ssr) / 2 / (reduced_results_X2.ssr / (len(df) - 4))
partial_F_X3 = (results.ssr - reduced_results_X3.ssr) / 2 / (reduced_results_X3.ssr / (len(df) - 4))

print(f"Частный F-критерий для X1: {partial_F_X1}")
print(f"Частный F-критерий для X2: {partial_F_X2}")
print(f"Частный F-критерий для X3: {partial_F_X3}")

NameError: name 'results' is not defined

In [ ]:
from scipy.stats import f
#рассчитаем критическое f
critical_value = f.ppf(1 - 0.05, 3, len(df) - 4)
critical_value

Таким образом, сравнивая частные Fфакт и Fкрит приходим к выводу о необходимости отклонить гипотезу (Н0) и с вероятностью 95% сделать вывод о статистической значимости уравнения регрессии и значения R^2, так как они статистически надежны и сформировались под систематическим действием неслучайных причин.

In [14]:
#выведем частные t-критерии
print(f"Частный t-критерий для X1: {partial_F_X1**0.5}")
print(f"Частный t-критерий для X2: {partial_F_X2**0.5}")
print(f"Частный t-критерий для X3: {partial_F_X3**0.5}")

Частный t-критерий для X1: 6.045719623412578
Частный t-критерий для X2: 6.278220781784259
Частный t-критерий для X3: 5.769733939409966


In [15]:
#найдем табличное t
from scipy.stats import t
t_critical_value = t.ppf(1 - 0.05/2, len(df) - 2)
t_critical_value

2.022690911734728

Значимость коэффициентов регрессии полученного уравнения можно оценить с помощью частных t-критериев Стьюдента. В полученной модели tтабл = 2,02.\
tb1>tтабл, (6,05>2.02)\
tb2>tтабл (6,28>2.02)\
tb3>tтабл (6,28>2.02)\
Таким образом, коэффициенты регрессии является статистически значимыми при включении в модель.

# Множественный коэффициент корреляции и скорректированный коэффициент детерминации

In [16]:
#рассчитаем множественный коэффициент корреляции
multicorr = results.rsquared

#рассчитайте скорректированный коэффициент детерминации
n = len(y)
k = len(independent_variables) + 1  # Количество параметров (включая константу)
adj_r2 = 1 - ((1 - multicorr) * (n - 1) / (n - k - 1))

# Выведите результаты
print(f"Множественный коэффициент корреляции (R): {multicorr}")
print(f"Скорректированный коэффициент детерминации (R^2): {adj_r2}")

Множественный коэффициент корреляции (R): 0.1330625735276486
Скорректированный коэффициент детерминации (R^2): 0.03673619280849849


Таким образом, множественный коэффициент корреляции R=0,13 говорит нам о слабой прямой связи. И всего 3,7% изменчивости ВВП на душу населения объясняется построенной моделью.

# Показатели частной корреляции

In [17]:
from scipy.stats import pearsonr
#обозначим переменные
X1 = 'Interest Rate'
X2 = 'Inflation Rate'
X3 = 'Jobless Rate'
Y = 'GDP/P'

#рассчитаем частную корреляцию между X1 и Y, учитывая X2 и X3
partial_corr_X1_Y, _ = pearsonr(df[X1] - df[X2] * (df[X1].corr(df[X2])) - df[X3] * (df[X1].corr(df[X3])),
                                df[Y] - df[X2] * (df[Y].corr(df[X2])) - df[X3] * (df[Y].corr(df[X3])))

#рассчитаем частную корреляцию между X2 и Y, учитывая X1 и X3
partial_corr_X2_Y, _ = pearsonr(df[X2] - df[X1] * (df[X2].corr(df[X1])) - df[X3] * (df[X2].corr(df[X3])),
                                df[Y] - df[X1] * (df[Y].corr(df[X1])) - df[X3] * (df[Y].corr(df[X3])))

#рассчитаем частную корреляцию между X3 и Y, учитывая X1 и X2
partial_corr_X3_Y, _ = pearsonr(df[X3] - df[X1] * (df[X3].corr(df[X1])) - df[X2] * (df[X3].corr(df[X2])),
                                df[Y] - df[X1] * (df[Y].corr(df[X1])) - df[X2] * (df[Y].corr(df[X2])))

# Выведите результаты
print("Частная корреляция между X1 и Y, учитывая X2 и X3:")
print(partial_corr_X1_Y)

print("Частная корреляция между X2 и Y, учитывая X1 и X3:")
print(partial_corr_X2_Y)

print("Частная корреляция между X3 и Y, учитывая X1 и X2:")
print(partial_corr_X3_Y)

Частная корреляция между X1 и Y, учитывая X2 и X3:
-0.050792005664470584
Частная корреляция между X2 и Y, учитывая X1 и X3:
-0.32291614939829405
Частная корреляция между X3 и Y, учитывая X1 и X2:
-0.3025276817330096


Таким образом, имеем:
1) слабую отрицательную (обратную) связь между ВВП на душу населения и процентной ставкой, при исключении других факторов. 
2) умеренную отрицательную (обратную) связь между ВВП на душу населения и инфляцией, при исключении других факторов. 
3) умеренную отрицательную (обратную) связь между ВВП на душу населения и безработицей, при исключении других факторов. 

# Модель с фиктивными переменными

Введем фиктивные переменные по политико-географическому местоположению: страны Евросоюза - 1, другие - 0.

In [18]:
#создадим список со странами Евросоюза
eu = ['Euro Area','Austria', 'France', 'Malta', 'Belgium', 'Germany', 'Netherlands', 'Bulgaria', 'Greece', 'Poland', 'Croatia', 'Hungary', 'Portugal', 'Cyprus', 'Ireland', 'Romania', 'Czech Republic','Italy','Slovakia','Denmark','Latvia','Slovenia','Estonia','Lithuania','Spain','Finland','Luxembourg','Sweden']

#создадим новый столбец с фиктивными переменными
df['Fict_eu'] = df['Country'].apply(lambda x: 1 if x in eu else 0)

In [33]:
#построим модель
#зависимая переменная
dependent_variable_f = 'GDP/P'
#факторы
independent_variables_f = ['Interest Rate', 'Inflation Rate', 'Jobless Rate', 'Fict_eu']

#добавим константы к факторам
X_f = sm.add_constant(df[independent_variables_f])

#выделим зависимую переменную
y_f = df[dependent_variable_f]

#создадим модель множественной регрессии
model_f = sm.OLS(y_f, X_f)

#получим результаты оценки модели
results_f = model_f.fit()

#выведем статистики оценки параметров
results_f.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  GDP/P   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.274
Method:                 Least Squares   F-statistic:                     4.775
Date:                Mon, 27 Nov 2023   Prob (F-statistic):            0.00341
Time:                        09:22:06   Log-Likelihood:                -188.18
No. Observations:                  41   AIC:                             386.4
Df Residuals:                      36   BIC:                             394.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             73.1930     13.017      5.623      0.000      46.793      99.593
Interest Rate     -2.0061      0.959     -2.092      0.044      -3.951      -0.061
Inflation Rate    -0.4954      0.377     -1.315      0.197      -1.259       0.269
Jobless Rate      -2.3398      0.889     -2.632      0.012      -4.143      -0.537
Fict_eu           -8.1352     10.090     -0.806      0.425     -28.598      12.328
==============================================================================
Omnibus:                       19.941   Durbin-Watson:                   1.880
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               30.277
Skew:                           1.400   Prob(JB):                     2.66e-07
Kurtosis:                       6.144   Cond. No.                         82.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

По полученному регрессионному анализу можем составить множественное уравнение регрессии: \
**y = -2,01x_1 - 0,5x_2 - 2,34x_3 - 8,14x_4 + 73,19**

# Исследование остатков на гетероскедастичность

## Без фиктивной переменной

In [20]:
from statsmodels.stats.diagnostic import het_goldfeldquandt
#тест Голдфельда-Квандта на гетероскедастичность
_, p_value, _ = het_goldfeldquandt(results.resid, X)

#результаты
print(f"P-value теста Голдфельда-Квандта: {p_value}")

#интерпретация результата
if p_value < 0.05:
    print("Отвергаем гипотезу о гомоскедастичности (принимаем гипотезу о гетероскедастичности).")
else:
    print("Принимаем гипотезу о гомоскедастичности.")

P-value теста Голдфельда-Квандта: 0.008269801073063378
Отвергаем гипотезу о гомоскедастичности (принимаем гипотезу о гетероскедастичности).


## С фиктивной переменной 

In [21]:
#тест Голдфельда-Квандта на гетероскедастичность
_, p_value, _ = het_goldfeldquandt(results_f.resid, X_f)

#результаты
print(f"P-value теста Голдфельда-Квандта: {p_value}")

#интерпретация результата
if p_value < 0.05:
    print("Отвергаем гипотезу о гомоскедастичности (принимаем гипотезу о гетероскедастичности).")
else:
    print("Принимаем гипотезу о гомоскедастичности.")

P-value теста Голдфельда-Квандта: 0.007321609883310612
Отвергаем гипотезу о гомоскедастичности (принимаем гипотезу о гетероскедастичности).


Таким образом, и в том и в другом случае, остатки гетероскедастичны.

# Уравнения регрессии 

## Линейное

In [188]:
#создадим списки для сравнения уравнений регрессии
rsquared = []
fstat = []
astat = []

In [189]:
#линейная модель
dependent_variable = 'GDP/P'
independent_variables = ['Interest Rate', 'Inflation Rate', 'Jobless Rate']
X = sm.add_constant(df[independent_variables])
y = df[dependent_variable]
model_l = sm.OLS(y, X)
results_l = model.fit()

#добавим данные в список
rsquared.append(results_l.rsquared)
fstat.append(results_l.fvalue)

In [190]:
#найдем ошибку аппроксимации
X = np.array(X).reshape(-1, 4)
y = np.array(y)
y_pred = results.predict(X)
a = abs((y - y_pred) / y).sum() / len(df) *100
astat.append(a)

## Гиперболическая

In [191]:
#удалим строку с нулем, так как она не дает поделить 1/x
df = df.loc[df['Interest Rate'] != 0]

dependent_variable = 'GDP/P'
independent_variables = ['Interest Rate', 'Inflation Rate', 'Jobless Rate']
X = sm.add_constant(df[independent_variables])
y = df[dependent_variable]

X = np.array(X).reshape(-1, 4)
y = np.array(y)

#построим модель
X_hyp = X**-1
model_hyp = sm.OLS(y, X_hyp)
results_hyp = model_hyp.fit()

In [192]:
#найдем нужные статистики
rsquared.append(results_hyp.rsquared)
fstat.append(results_hyp.fvalue)

#найдем ошибку аппроксимации
y_pred = results_hyp.predict(X_hyp)
a = abs((y - y_pred) / y).sum() / len(df) *100
astat.append(a)

In [222]:
results_hyp.tvalues

array([-0.58215015,  1.24606154,  2.39290378,  1.22702974])

Все коэффициенты, кроме b2 незначимы

## Полулогарифмическая 

In [198]:
#построим модель
X_log = np.log(X)
model_log = sm.OLS(y, X_log)
results_log = model_log.fit()

In [199]:
#найдем нужные статистики
rsquared.append(results_log.rsquared)
fstat.append(results_log.fvalue)

#найдем ошибку аппроксимации
y_pred = results_log.predict(X_log)
a = abs((y - y_pred) / y).sum() / len(df) *100
astat.append(a)

In [223]:
results_log.tvalues

array([        nan, -3.44070236,  3.19793577, -0.63456635])

Все коэффициенты, кроме b3 значимы

## Степенная 

In [203]:
#построим модель
X_log = np.log(X)
y_log = np.log(y)
model_log_2 = sm.OLS(y_log, X_log)
results_log_2 = model_log_2.fit()

In [227]:
#найдем нужные статистики
rsquared.append(results_log_2.rsquared)
fstat.append(results_log_2.fvalue)

#найдем ошибку аппроксимации
y_pred = results_log_2.predict(X_log)
a = abs((y - y_pred) / y).sum() / len(df) *100
astat.append(a)

In [224]:
results_log_2.tvalues

array([        nan, -4.56587606,  5.72018908, -0.09733935])

Все коэффициенты, кроме b3 значимы

## Экспоненциальная 

In [208]:
#построим модель
y_log = np.log(y)
model_exp = sm.OLS(y_log, X)
results_exp = model_exp.fit()

In [209]:
#найдем нужные статистики
rsquared.append(results_exp.rsquared)
fstat.append(results_exp.fvalue)

#найдем ошибку аппроксимации
y_pred = results_exp.predict(X)
a = abs((y - y_pred) / y).sum() / len(df) *100
astat.append(a)

In [225]:
results_exp.tvalues

array([23.67409585, -4.65988681, -0.96885598, -4.37387074])

Все коэффициенты, кроме b2 значимы

In [216]:
comp = pd.DataFrame({'Уравнение': ['Линейное', 'Гиперболическое', 'Полулогарифмическое', 'Степенное', 'Экспоненциальное'], 'R-квадрат': rsquared, 'F-статистика': fstat, 'Ошибка А, %': astat})
comp

,Уравнение,R-квадрат,F-статистика,"Ошибка А, %"
0,Линейное,0.334836,6.208447,92.999220
1,Гиперболическое,0.448466,9.757509,70.759923
2,Полулогарифмическое,0.598166,18.359249,83.378114
3,Степенное,0.894731,104.827120,83.942133
4,Экспоненциальное,0.616060,19.254868,84.824034


Видим, что по следующим статистикам лучшие уравнения регрессии:
1) R-квадрат: степенное
2) F-статистика: степенное
3) Ошибка А: гиперболическое
Также, коэффициенты значимы в степенном и экспоненциальном уравнении. Из всех приведенных фактов делаем вывод, что степенное уравнение регрессии самое лучшее.

## Точечный прогноз 

In [237]:
results_log_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.895
Model:                            OLS   Adj. R-squared (uncentered):              0.886
Method:                 Least Squares   F-statistic:                              104.8
Date:                Mon, 27 Nov 2023   Prob (F-statistic):                    3.84e-18
Time:                        12:02:39   Log-Likelihood:                         -60.513
No. Observations:                  40   AIC:                                      127.0
Df Residuals:                      37   BIC:                                      132.1
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const               0          0        nan        nan           0           0
x1            -1.0555      0.231     -4.566      0.000      -1.524      -0.587
x2             1.6120      0.282      5.720      0.000       1.041       2.183
x3            -0.0327      0.336     -0.097      0.923      -0.714       0.648
==============================================================================
Omnibus:                        0.016   Durbin-Watson:                   1.791
Prob(Omnibus):                  0.992   Jarque-Bera (JB):                0.073
Skew:                          -0.014   Prob(JB):                        0.964
Kurtosis:                       2.793   Cond. No.                          inf
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

Уравнение регрессии: ln(y) = -1,0555*ln(x_1) + 1,612*ln(x_2) - 0,0327*ln(x_3)\
или **y = x1^-1,0555 * x2^1,612 * x3^-0,0327**

In [234]:
import matplotlib.pyplot as plt
y_pred = results_log_2.predict(X_log)
y_pred

array([3.41444399, 3.42040936, 2.7986073 , 2.47604816, 3.22112865,
       2.04883015, 3.22376168, 4.03145797, 4.43503241, 2.63516566,
       2.51913708, 3.02674227, 3.61139807, 3.20397573, 2.21858831,
       3.49788919, 4.13388687, 2.97156686, 2.47169788, 2.56437077,
       3.30225862, 3.6882402 , 1.6880813 , 2.08982172, 3.96744597,
       2.8267184 , 3.01807331, 1.93728846, 4.82371913, 2.7193784 ,
       3.41981982, 4.700005  , 4.80962799, 3.18869105, 1.69929032,
       3.0757783 , 2.43736766, 2.86639993, 2.41610482, 3.47365944])